# Imports für das Notebook

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
from sklearn.metrics import recall_score

# Vorbereitung der Daten

In [4]:
# Einlesen der gesäuberten Daten für Exploration und Visualisierung
heart_data = pd.read_csv("../../resources/data_clean/heart_2020_clean.csv")

# Filtern von zu hohen Schlafzeiten
heart_data = heart_data[heart_data["SleepTime"] <= np.percentile(heart_data["SleepTime"], 99.9)]

# Trainieren des XGBoost-Modells
model_xgb = xgb.XGBClassifier()

# Konvertierung der kategorialen Attribute
labelencoder = LabelEncoder()

heart_data["AgeCategory"] = labelencoder.fit_transform(heart_data["AgeCategory"])
heart_data["KidneyDisease"] = heart_data["KidneyDisease"].map({"Ja": 1, "Nein": 0})
heart_data["Stroke"] = heart_data["Stroke"].map({"Ja": 1, "Nein": 0})
heart_data["GenHealth"] = labelencoder.fit_transform(heart_data["GenHealth"])

# Standardisierung der numerischen Attribute
scaler = StandardScaler()
heart_data["SleepTime"] = scaler.fit_transform(heart_data["SleepTime"].values.reshape(-1, 1))

Die CSV-Datei wird hier so angepasst, dass der nächste Abschnitt sich um das Modell für die Vorhersage kümmern kann.  
Es werden hier die nichtaussagekräftigen Werte aus dem Datensatz gefiltert und kategoriellen Variablen werden umcodiert, sodass sie vom Modell verarbeitet werden können

# XGBoost-Modell

In [5]:
# Trennen von Trainings- und Testdatensatz
X = heart_data[["AgeCategory", "SleepTime", "KidneyDisease", "Stroke", "GenHealth"]]
y = heart_data["HeartDisease"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Weitere Kategorisierung
y_train_encoded = labelencoder.fit_transform(y_train)
y_test_encoded = labelencoder.fit_transform(y_test)

# Training des Modells
model_xgb.fit(X_train, y_train_encoded)

# Vorhersagen auf dem Testdatensatz
y_pred_xgb = model_xgb.predict(X_test)

# Bewertung der Modellleistung anhand des Recalls
recall_xgb = recall_score(y_test_encoded, y_pred_xgb, pos_label=1)

print("XGBoost Recall (Ja):", recall_xgb)

XGBoost Recall (Ja): 0.9958928724223496


Unsere Modelle basieren auf den 5 wichtigsten Attributen, die auf eine Herzerkrankung hinweisen können. Diese wurden im anderen Notebook in diesem Ordner manuell identifiziert.  
Der Recall, der misst, wie viele der tatsächlich Erkrankten vom Modell korrekt identifiziert werden, liegt in unserem Fall bei herausragenden 99,6%. Dies bedeutet, dass die meisten Vorhersagen des Modells korrekt eine Herzerkrankung erkennen.